# Курсовая работа
## по дисциплине "Структуры и алгоритмы обработки данных"
## Выполнила студентка группы БВТ2103 Пьянова Анна Олеговна

### Задание

Написать программу, которая будет считывать данные из CSV файла, содержащего
информацию о продажах товаров в магазине. Данные в файле содержатся в следующем
формате:
| Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена
за единицу | Общая стоимость |

Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве проданных единиц каждого товара и доле каждого товара в общей выручке.

Для решения задач необходимо использовать структуры данных, такие как массивы и хеш-таблицы, а также различные алгоритмы обработки данных, например, сортировку и поиск. Также необходимо учитывать возможные ошибки ввода-вывода и обрабатывать их в
соответствии с требованиями.

In [22]:
#при несоответствии ключей таблицы требованиям в условии
class KeysNoEqualError(Exception):

    def __init__(self, keys, message = "Keys don't match!"):
        self.keys = keys
        self.message = message
        super().__init__(self.message)


#при ошибочных значениях у поступивших на вход данных
class ValueInputError(Exception):
    
    def __init__(self, message = "Values don't match the format!"):
        self.message = message
        super().__init__(self.message)

In [23]:
check_keys = [
        "Номер заказа", 
        "Дата заказа",
        "Название товара",
        "Категория товара",
        "Количество продаж",
        "Цена за единицу",
        "Общая стоимость"
    ]

def check_values(item):
    #проверка на положительные числовые значения
    if (int(item["Номер заказа"]) < 0 and int(item["Количество продаж"]) < 0 
        and int(item["Цена за единицу"]) < 0 and int(item["Общая стоимость"]) < 0):
        return False
    date = item["Дата заказа"]
    #проверка на верный формат даты
    if 1 > int(date[:2]) > 31 and 1 > int(date[3:5]) > 12 and 1990 > int(date[6:]) > 2023:
        return False
    return True


In [24]:
import csv

def open_csv(file_path):   
 
    #преобразование элементов таблицы в словарь
    with open(file_path, encoding='utf-8') as r_file:
        file_reader = csv.DictReader(r_file, delimiter = ";")
        items = []
        for row in file_reader:
            if not check_values(row):
                raise ValueInputError
            items.append(row)

    #получение списка ключей (названий столбцов) таблицы
    with open('table.csv', 'r', encoding='utf-8', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';')
        for row in spamreader:
            keys = row
            break

    #обработка исключения
    if keys != check_keys:
        raise KeysNoEqualError(keys)
    
    return items, keys

items, keys = open_csv("table.csv")

### Задание 1

In [25]:
sum = 0

for item in items:
    sum += int(item["Общая стоимость"])

print("Общая выручка магазина:", sum)

Общая выручка магазина: 241969500


### Задание 2

In [26]:
#быстрая сортировка
def quick_sort(array, key):
    if len(array) <= 1:
        return array
    else:
        barrier = int(array[len(array) // 2][key])
        left = [i for i in array if int(i[key]) < barrier]
        middle = [i for i in array if int(i[key]) == barrier]
        right = [i for i in array if int(i[key]) > barrier]
        
    return quick_sort(left, key) + middle + quick_sort(right, key)

In [27]:
print("Характеристики товара, проданного наибольшее количество раз:")
quick_sort(items, "Количество продаж")[-1]

Характеристики товара, проданного наибольшее количество раз:


{'Номер заказа': '4010',
 'Дата заказа': '10.02.2022',
 'Название товара': 'Philips Hue Smart Bulbs',
 'Категория товара': 'Умный дом',
 'Количество продаж': '500',
 'Цена за единицу': '2990',
 'Общая стоимость': '1495000'}

### Задание 3

In [28]:
#реализация Дека
class Deque:

    #инициализация
    def __init__(self):
        self.items = []

    #проверка на пустоту
    def isEmpty(self):
        return self.items == []

    #добавление нового элемента в конец
    def add(self, item):
        self.items.append(item)

    #добавление нового элемента в начало
    def addLeft(self, item):
        self.items.insert(0,item)

    #извлечение элемента из конца
    def pop(self):
        return self.items.pop()

    #извлечение элемента из начала
    def popLeft(self):
        return self.items.pop(0)
    
    #возвращение последнего элемента
    def get(self):
        return self.items[len(self.items) - 1]

    #возвращение количества элементов
    def lenght(self):
        return len(self.items)

In [29]:
#сортировка по возрастанию с использованием двух Деков
def sort_with_deque(array, key):
    noSort = Deque()
    sort = Deque()

    for i in array:
        noSort.add(i)

    while not noSort.isEmpty():
        cur = noSort.pop()
        while not sort.isEmpty() and int(sort.get()[key]) > int(cur[key]):
            noSort.addLeft(sort.pop())
        sort.add(cur)

    return sort.get()

In [30]:
print("Характеристики товара, который принёс наибольшую выручку:")
sort_with_deque(items, "Общая стоимость")

Характеристики товара, который принёс наибольшую выручку:


{'Номер заказа': '4001',
 'Дата заказа': '01.02.2022',
 'Название товара': 'Apple iPhone 14',
 'Категория товара': 'Смартфоны',
 'Количество продаж': '500',
 'Цена за единицу': '89990',
 'Общая стоимость': '44995000'}

### Задание 4

In [31]:
print("Общая выручка магазина:", sum)

for item in quick_sort(items, "Общая стоимость"):
    try:
        print(
            "Название товара:", item["Название товара"], ";",
            "Количество продаж:", item["Количество продаж"], ";",
            "Доля товара в общей выручке:", int(item["Общая стоимость"]) / sum * 100, "%"
        )
    except ZeroDivisionError:
        print("Can't divide by zero!")
        break


Общая выручка магазина: 241969500
Название товара: Philips Hue Smart Bulbs ; Количество продаж: 500 ; Доля товара в общей выручке: 0.6178464641204781 %
Название товара: Philips Sonicare DiamondClean ; Количество продаж: 200 ; Доля товара в общей выручке: 0.8257239032192073 %
Название товара: Nest Learning Thermostat ; Количество продаж: 100 ; Доля товара в общей выручке: 0.8261371784460437 %
Название товара: Bose SoundLink Revolve+ ; Количество продаж: 100 ; Доля товара в общей выручке: 1.032774791864264 %
Название товара: Bose QuietComfort 55 ; Количество продаж: 100 ; Доля товара в общей выручке: 1.446050018700704 %
Название товара: DJI Mavic 3 ; Количество продаж: 25 ; Доля товара в общей выручке: 1.5496787818299413 %
Название товара: Samsung Galaxy Watch 5 ; Количество продаж: 150 ; Доля товара в общей выручке: 1.8591186079237259 %
Название товара: LG UltraFine ; Количество продаж: 50 ; Доля товара в общей выручке: 2.0661694965687825 %
Название товара: Samsung Galaxy Tab S8 ; Колич